# Simulating fly-bys

This notebook will take you through a simulation of fly-bys of a star perturbing a planetary system.

### Imports

In [1]:
import rebound
import numpy as np

from IPython.display import display, clear_output
import matplotlib.pyplot as plt

AttributeError: 'tuple' object has no attribute 'type'

### Setup
Let's first create a simulation of our target planetary system. We'll use Sun and the outer planets of our solar system for now. Using NASA Horizons database to get the details of each object.

In [ ]:
sim = rebound.Simulation()
particle_names = ["Sun", "Jupiter", "Saturn", "Uranus", "Neptune"]
# we use the NASA horizon database to look up the Sun and planets
sim.add(particle_names)

# let's give all the particles a unique hash (based on its name)
for i, particle in enumerate(sim.particles):
        particle.hash = particle_names[i]

sim.status()
    

Let's save this simulation for future reference.

In [ ]:
sim.save("solar_system_outer_planets.bin")

In [ ]:
sim = rebound.Simulation.from_file("solar_system_outer_planets.bin")

Let's define a function that simulates a single fly-by given a certain system and a particle that will be intruding the system.

In [ ]:
def simulate_fly_by(sim, intruder, visualize=False):
    intruder.hash = "intruder"
    
    sim.add(intruder)
    
    intruder_distance = np.linalg.norm(sim.particles["intruder"].xyz)
    sim.exit_max_distance = intruder_distance*1.01
    
    while True:
        try:
            sim.integrate(sim.t+5)
            
            if visualize:
                fig = rebound.OrbitPlot(sim,color=True,unitlabel="[AU]")
                display(fig)
                plt.close(fig)
                clear_output(wait=True)

        except rebound.Escape as error:
            #print(error)
            for i, particle in enumerate(sim.particles):
                distance = np.linalg.norm(particle.xyz)
                if distance > sim.exit_max_distance:
                    #print("Removed", i, str(particle.hash))
                    sim.remove(hash=particle.hash)
                    sim.move_to_com()
                    
            return sim


In [ ]:
def calc_escape_velocity(sim, particle):
    #sim.move_to_hel()
    
    r = np.linalg.norm(particle.xyz)
    G = sim.G
    m = sim.particles[0].m
    
    return np.sqrt(2 * G * m / r)

In [ ]:
def strong_regime(resolution=100, n_trials=50):
    print("Starting strong regime simulation with resolution {}, {} trials each...".format(resolution, n_trials))
    xs = np.linspace(1, 50, resolution)
    f_eject = np.ones(resolution)
    
    for i, x in enumerate(xs):
        print("Running r_min =", x)
        eject_count = 0.
        
        # run n_trials trials detecting ejection directly after fly-by
        for j in range(n_trials):
            # get a fresh simulation
            sim = rebound.Simulation.from_file("solar_system_outer_planets.bin")
            sim = randomize_sim(sim)
            
            intruder = rebound.Particle(m=1.,x=x,y=-1000.,vy=2.)
            
            sim = simulate_fly_by(sim, intruder)
            
            sim.move_to_hel()
            for particle in sim.particles:
                v = np.linalg.norm(particle.vxyz)
                v_esc = calc_escape_velocity(sim, particle)
                if v > v_esc:
                    eject_count += 1
                    break
        print("Detected", eject_count, "ejections out of", n_trials, "trials.")
        f_eject[i] = eject_count / n_trials
        print(f_eject[i])

    
    return (xs, f_eject)            

In [3]:
# function to check distances, returning 'True' if planets near eachother by mutual Hill radius
import sys
from scipy import spatial


def mutual_rhill(p1, p2):
    """
    Calculates mutual Hill radius of particle 1 and 2.
    """
    r_hill_m = (p1.a + p2.a) / 2. * ((p1.m + p2.m) / 3.)**(1/3.)
    return r_hill_m
    


def check_orbit_crossing(simulation):
    """
    Checks if orbits cross.
    """
#     make distance matrix
    locations = []
    for particle in simulation.particles:
        locations.append(particle.xyz)
    dist_matrix = spatial.distance.pdist(locations, metric='euclidian')
    
#     only checking all values underneath diagonal, comparing to mutual r_hill
    for i in range(len(simulation.particles)):
        for j in range(i):
            p1 = simulation.particles[i]
            p2 = simulation.particles[j]
            if mutual_rhill(p1, p2) >= dist_matrix[i][j]:
                return True
            
    return False

ImportError: cannot import name _distributor_init

Let's define a function to predict the stabillity of a system directly after a fly-by.

Instabillity can be defined in a number of ways. The simplest being direct ejection from the system.

This function will try to analyze the stability of a system based on direct observations of it's properties.

In [ ]:
def analyze_stability(sim):
    
    if check_immediate_ejection(sim) == True:
        return False
    
    elif check_orbit_crossing(sim) == True:
        return False
    
    elif check_kozai(sim) == True:
        return False
    
    elif check_AMD(sim) == True:
        return False
    
    else:
        return True
    
    
    
    
    

In [ ]:
def check_immediate_ejection(sim):
    # move to Sun frame
    sim.move_to_hel()
    
    # calculate velocity of each particle and compare to escpae velocity
    for particle in sim.particles:
        v = np.linalg.norm(particle.vxyz)
        v_esc = calc_escape_velocity(sim, particle)
        if v >= v_esc:
            return True
    
    return False
    


In [ ]:
xs, f_eject = strong_regime(resolution=30, n_trials=5000)

plt.plot(xs, f_eject)

In [ ]:
plt.plot(xs, 1-f_eject)

In [ ]:
a = np.ones(10)
a[2] = 5
a


In [ ]:
def randomize_sim(sim):
    sim.integrate(np.random.random()*10**3)
    return sim
    

In [ ]:
sim = rebound.Simulation.from_file("solar_system_outer_planets.bin")
intruder = rebound.Particle(m=1.,x=x,y=-1000.,vy=2.)
simulate_fly_by(sim, intruder, visualize=True)